In [7]:
## Libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


## Plot settings
plt.rc('xtick', direction='in', labelsize=14)
plt.rc('ytick', direction='in', labelsize=14)
plt.rc('axes', labelsize=20, titlesize=22)
plt.rc('legend', fontsize=14)
plt.rc('animation', html='jshtml')
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Computer Modern Roman']
plt.rcParams['mathtext.fontset'] = 'cm'

In [8]:
## Read coordinates and indices

grid_coordinates = []
with open('../../../../Grids/AB_grid.txt', 'r') as reader:
    line = reader.readline()

    while line != '':
        elements = line.split(',')
        point = (float(elements[0]),float(elements[1]))
        grid_coordinates.append(point)
        line = reader.readline()

jump_coordinates = []
with open('../../../../Grids/AB_jump_coordinates.txt', 'r') as reader:
    line = reader.readline()

    while line != '':
        elements = line.split(' ')
        data_block = []
        for ele in elements:
            temp = ele.replace('(', '').replace(')', '')
            if temp != 'stop':
                x = temp.split(',')[0]
                y = temp.split(',')[1]
                point = (float(x),float(y))
                data_block.append(point)
            else:
                break
        line = reader.readline()
        jump_coordinates.append(data_block)

jump_indices = []
with open('../../../../Grids/AB_jump_indices.txt', 'r') as reader:
    line = reader.readline()

    while line != '':
        elements = line.split(' ')
        data_block = []
        for ele in elements:
            if ele != 'stop':
                data_block.append(int(ele))
            else:
                break
        line = reader.readline()
        jump_indices.append(data_block)

bott_indices = []
with open('../../AB_Bott_indices_B_scale_0250.txt', 'r') as reader:
    line = reader.readline()
    
    while line != '':
        ele = line.replace('(', '').replace(')', '')
        elements = ele.split(',')
        package = (float(elements[0]), int(elements[1]))
        line = reader.readline()
        bott_indices.append(package)

# For drawing vertices use grid_coordinates
# For drawing lines use jump_coordinates
# For indices use jump_indices

## We now want to scale down the plot so it will fit inside a unit square in the 1st quadrant

# Compare x and y components seperatly to for the maximal distances along each axis
xs = [x for x,y in grid_coordinates]
ys = [y for x,y in grid_coordinates]

xs_length = abs(max(xs)) + abs(min(xs))
ys_length = abs(max(ys)) + abs(min(ys))

# We can now scale all xs and ys to fit inside a unit square
xs_scaled = [i/xs_length for i in xs]
ys_scaled = [i/ys_length for i in ys]
grid_scaled = [(x,y) for x, y in zip(xs_scaled, ys_scaled)]

# Same thing for all jump coodinates
jump_scaled = []
for i in jump_coordinates:
    xs_temp = [j[0]/xs_length for j in i]
    ys_temp = [j[1]/ys_length for j in i]
    jump_temp = [(x,y) for x, y in zip(xs_temp, ys_temp)]
    jump_scaled.append(jump_temp)


# The new vertices are stored in grid_scaled
# The new jumps are stored in jump_scaled

## IMPORTANT WE WILL USE THE GRID SCALED FOR ROTATIONS THIS MAKES THE WORK EASIER ##

# Read the phases in
evec_phases = []
with open('../../../../Separate_files/AB_args.txt', 'r') as reader:
    line = reader.readline()

    while line != '':
        ele = line.strip('\n')
        val = float(ele)
        evec_phases.append(val)
        line = reader.readline()

phases_ind = [(ele, count) for count, ele in enumerate(evec_phases)]

# Time evolution by giving index of states
def time_evolve_evec(group_of_states, time, evec, eval):

    hbar = 1
    t = time

    # Make the group of states in to vectors and multiply with time op
    phis = [np.exp(-1j*(eval[i]/hbar)*t)*evec[i] for i in group_of_states]

    # Make a linear combination equally divided
    a = len(phis)
    psi = sum([(1/a)*x for x in phis])

    phi_norm = psi / (np.sqrt(np.vdot(psi,psi)))

    density_of_states = [(x.conj() * x).real for x in phi_norm]
    
    return phi_norm, density_of_states


In [9]:
# Now load in the different eigenvalues and eigenvectors
pure_evals = np.loadtxt("../Eigenvalues/pure.txt")
un_mixed_evals = np.loadtxt("../Eigenvalues/unpure_mixed.txt")
un_positive_evals = np.loadtxt("../Eigenvalues/unpure_positive.txt")

pure_evecs = np.loadtxt("../Eigenvectors/pure.txt", dtype= np.complex128)
un_mixed_evecs = np.loadtxt("../Eigenvectors/unpure_mixed.txt", dtype= np.complex128)
un_positive_evecs = np.loadtxt("../Eigenvectors/unpure_positive.txt", dtype= np.complex128)

In [10]:
"""# Now define the group of indices of interest. They stem from analysis script
group = 542, 544, 545, 546, 547

## Magic line for animation
%matplotlib inline
# Attempt at animating time evolution

# Set figure up first
fig, ax = plt.subplots(figsize=(6,6))
# Set bascis
sites = ax.scatter(x=[], y=[], c=[])

xs = [w[0] for w in grid_scaled]
ys = [w[1] for w in grid_scaled]

evals = [pure_evals, un_mixed_evals, un_positive_evals]
evecs = [pure_evecs, un_mixed_evecs, un_positive_evecs]
type = ['pure', 'mixed', 'positive']

for a, b, c in zip(evals, evecs, type):
    # Set timesteps, state of choice for animation, and number of frames
    timestep = 1
    n_frames = 600
    B_scale = 0.25

    psi_0, dos_0 = time_evolve_evec(group, 0, b, a)

    dos_max = max(dos_0)

    plt.set_cmap('hot_r')
    #cax = ax.scatter(x= xs, y= ys, s= marker_size, c= dos_max, zorder= 2)


    # This function will do the animating
    def update(i):
        plt.clf()
        # t is the steps we progress, timestep sets the steplength
        t = i * timestep

        # Using the time evolution to find density of states for each increment
        psi, dos = time_evolve_evec(group, t, b, a)
        cf = plt.scatter(x= xs, y= ys, c= dos, vmin=0, vmax= dos_max)
        plt.plot([],[], linestyle= '', label= f'B scale = {B_scale}')
        plt.title(f'Timestep= {timestep} and Frames= {n_frames}')
        plt.legend(loc= 3)
        plt.colorbar(cf)
        plt.axis('equal')


    anim = FuncAnimation(fig, update, frames= n_frames, interval=250)

    anim.save(f'Time_evolutions/' + c + '.gif', writer='imagemagick', fps=30)

    #anim"""

"# Now define the group of indices of interest. They stem from analysis script\ngroup = 542, 544, 545, 546, 547\n\n## Magic line for animation\n%matplotlib inline\n# Attempt at animating time evolution\n\n# Set figure up first\nfig, ax = plt.subplots(figsize=(6,6))\n# Set bascis\nsites = ax.scatter(x=[], y=[], c=[])\n\nxs = [w[0] for w in grid_scaled]\nys = [w[1] for w in grid_scaled]\n\nevals = [pure_evals, un_mixed_evals, un_positive_evals]\nevecs = [pure_evecs, un_mixed_evecs, un_positive_evecs]\ntype = ['pure', 'mixed', 'positive']\n\nfor a, b, c in zip(evals, evecs, type):\n    # Set timesteps, state of choice for animation, and number of frames\n    timestep = 1\n    n_frames = 600\n    B_scale = 0.25\n\n    psi_0, dos_0 = time_evolve_evec(group, 0, b, a)\n\n    dos_max = max(dos_0)\n\n    plt.set_cmap('hot_r')\n    #cax = ax.scatter(x= xs, y= ys, s= marker_size, c= dos_max, zorder= 2)\n\n\n    # This function will do the animating\n    def update(i):\n        plt.clf()\n        

In [12]:
for t in np.arange(0,30,5):
    # Set figure up first
    fig, ax = plt.subplots(figsize=(6,6))

    # Plot all possible paths between vertices
    for i,j in zip(grid_scaled, jump_scaled):
        xs = [w[0] for w in j]
        ys = [w[1] for w in j]
        for l,k in zip(xs,ys):
            ax.plot([i[0], l], [i[1], k], color= 'k', zorder= 1, linewidth= 0.5)

    # Get points for scatter plot
    xs = [w[0] for w in grid_scaled]
    ys = [w[1] for w in grid_scaled]

    # Time evolution segment
    group = [542, 544, 545, 546, 547]
    B_scale = 0.25
    # Initial setup
    psi_0, dos_0 = time_evolve_evec(group, 0,pure_evecs, pure_evals)
    dos_max = max(dos_0)

    # Actual evolution
    time = t
    psi, dos = time_evolve_evec(group, time, pure_evecs, pure_evals)

    # Plotting part
    cf = plt.scatter(x= xs, y= ys, c= dos, vmin=0, vmax= dos_max)
    plt.colorbar(cf)
    plt.axis('equal')

    plt.set_cmap('hot_r')
    plt.savefig(f'TE_frames/pure/AB_edge_time_{t}', bbox_inches= 'tight')
    plt.close()